In [1]:
import numpy as np
import pandas as pd
from deep_translator import GoogleTranslator

/home/kumarsau/venvs/jenv/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
fp = "/home/kumarsau/private/capula/pyhelp/names.csv"

In [3]:
df = pd.read_csv(fp)

In [4]:
en_names = {}
for x in df['shareholder_name'].unique():
    en_names[x] = GoogleTranslator(source='auto', target='english').translate(x)
df['en_shareholder_name'] = df['shareholder_name'].apply(lambda x: en_names.get(x))

In [5]:
from math import floor, ceil
 
def string_similarity_score(s1, s2):
    if (s1 == s2):
        return 1.0
    # Length of two s
    len1, len2 = len(s1), len(s2)
    # Maximum distance upto which matching is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
    # Count of matches
    match = 0
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first
    for i in range(len1):
        # Check if there is any matches
        for j in range(max(0, i - max_dist), 
                       min(len2, i + max_dist + 1)):
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
 
    # If there is no match
    if (match == 0):
        return 0.0
 
    # Number of transpositions
    t , point = 0, 0
    for i in range(len1):
        if (hash_s1[i]): 
            while (hash_s2[point] == 0):
                point += 1
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
 
    return (match/ len1 + match / len2 + (match - t) / match)/ 3.0

In [6]:
def sanitize_en(name):
    ignore_chars = ".,()!|:"
    name = name.lower()
    name = name.translate(str.maketrans('','', ignore_chars)).strip().split(" ")
    name = [x for x in name if x]
    name = name.translate(str.maketrans('０１２３４５６７８９', '0123456789'))
    return name

def optimal_string_alignment_distance(names1, names2):
    tsc = 0
    l1, l2 = len(names1), len(names2)
    for i in range(l1):
        sc = 0
        for j in range(max(0, i-3), min(l2, i+3)): # search in neighbouring string 
            sc = max(sc, string_similarity_score(names1[i], names2[j]))
        tsc += sc
        # print(f"names1[i] = {names1[i]}, sc = {sc}")
    return (tsc/max(len(names1), len(names2)))*min(len(names1), len(names2))/max(len(names1), len(names2))

def get_similarity_score(name1, name2):
    ignore_chars = ".,()!|:"
    name1 = sanitize_en(name1)
    name2 = sanitize_en(name2)
    return optimal_string_alignment_distance(name1, name2)

In [7]:
env = en_names.values()
print("\n".join(env))

The Master Trust Bank of Japan, Ltd. (Trust Account)
Japan Trustee Services Bank, Ltd. (Trust Account)
Japan Trustee Services Bank, Ltd. (Trust Account 5)
JP MORGAN CHASE BANK 385151 (standing proxy: Mizuho Bank, Ltd.)
NORTHERN TRUST CO. (AVFC) RE SILCHESTER INTERNATIONAL INVESTORS INTERNATIONAL VALUE EQUITY TRUST (Permanent agent: Hong Kong and Shanghai Bank Tokyo Branch)
Japan Trustee Services Bank, Ltd. (Trust Account 7)
STATE STREET BANK WEST CLIENT-TREATY 505234 (standing proxy: Mizuho Bank, Ltd.)
NORTHERN TRUST CO. (AVFC) RE U.S. TAX EXEMPTED PENTION FUNDS (Permanent agent: Hong Kong and Shanghai Banking Corporation, Tokyo Branch)
Japan Trustee Services Bank, Ltd. (Trust Account 1)
SSBTC CLIENT OMNIBUS ACCOUNT (Permanent Agent: Hong Kong and Shanghai Bank Tokyo Branch)
top_10_total
treasury
total
Custody Bank of Japan Ltd. (Trust Account)
SMBC Nikko Securities Inc.
Custody Bank of Japan Ltd. (Trust Account 5)
JP MORGAN CHASE BANK 385781 (standing proxy: Mizuho Bank, Ltd.)
Custody

In [8]:
orig_strings = {}
thr = 0.75
# to be read from hardcoded files
hardcoded_similar = []
hardcoded_dissimilar = []

for n in env:
    # print(f"\n n = ||{n}||\n")
    if len(orig_strings)==0:
        orig_strings[n] = [n]
    else:
        init_score = 0
        best_match_string = ""
        for k in orig_strings.keys():
            cur_score = max(init_score, get_similarity_score(n, k))
            if init_score < cur_score:
                init_score = cur_score
                best_match_string = k
            # print(f"n = |{n}|, init_score = |{init_score}|, best_match_string = |{best_match_string}|, k = |{k}|")

        # print(f"init_score = {init_score}, threshold = {thr}")
        if init_score > thr:
            orig_strings[k].append(n)
        else:
            orig_strings[n] = [n]

In [9]:
print("\n".join(orig_strings.keys()))

The Master Trust Bank of Japan, Ltd. (Trust Account)
Japan Trustee Services Bank, Ltd. (Trust Account)
JP MORGAN CHASE BANK 385151 (standing proxy: Mizuho Bank, Ltd.)
NORTHERN TRUST CO. (AVFC) RE SILCHESTER INTERNATIONAL INVESTORS INTERNATIONAL VALUE EQUITY TRUST (Permanent agent: Hong Kong and Shanghai Bank Tokyo Branch)
STATE STREET BANK WEST CLIENT-TREATY 505234 (standing proxy: Mizuho Bank, Ltd.)
SSBTC CLIENT OMNIBUS ACCOUNT (Permanent Agent: Hong Kong and Shanghai Bank Tokyo Branch)
top_10_total
treasury
total
SMBC Nikko Securities Inc.
Custody Bank of Japan Ltd. (Trust Account 5)
J.P. Morgan Securities Japan Co., Ltd.
THE BANK OF NEW YORK MELLON AS DEPOSITARY BANK FOR DR HOLDERS (Permanent agent: Mitsubishi UFJ Bank, Ltd.)
Nomura Group Employee Stock Ownership Plan
GOVERNMENT OF NORWAY (Standing proxy: Citibank N.A. Tokyo Branch)


In [10]:
for k in orig_strings.keys():
    print(f"key = {k}")
    print(f"values = {orig_strings[k]}")

key = The Master Trust Bank of Japan, Ltd. (Trust Account)
values = ['The Master Trust Bank of Japan, Ltd. (Trust Account)']
key = Japan Trustee Services Bank, Ltd. (Trust Account)
values = ['Japan Trustee Services Bank, Ltd. (Trust Account)', 'Japan Trustee Services Bank, Ltd. (Trust Account 5)']
key = JP MORGAN CHASE BANK 385151 (standing proxy: Mizuho Bank, Ltd.)
values = ['JP MORGAN CHASE BANK 385151 (standing proxy: Mizuho Bank, Ltd.)']
key = NORTHERN TRUST CO. (AVFC) RE SILCHESTER INTERNATIONAL INVESTORS INTERNATIONAL VALUE EQUITY TRUST (Permanent agent: Hong Kong and Shanghai Bank Tokyo Branch)
values = ['NORTHERN TRUST CO. (AVFC) RE SILCHESTER INTERNATIONAL INVESTORS INTERNATIONAL VALUE EQUITY TRUST (Permanent agent: Hong Kong and Shanghai Bank Tokyo Branch)', 'Japan Trustee Services Bank, Ltd. (Trust Account 7)']
key = STATE STREET BANK WEST CLIENT-TREATY 505234 (standing proxy: Mizuho Bank, Ltd.)
values = ['STATE STREET BANK WEST CLIENT-TREATY 505234 (standing proxy: Mizuho B

In [12]:
get_similarity_score("NORTHERN TRUST CO. (AVFC) RE U.S. TAX EXEMPTED PENTION FUNDS (Permanent agent: Hong Kong and Shanghai Banking Corporation, Tokyo Branch)",
                     "NORTHERN TRUST CO. (AVFC) RE U. S. TAX EXEMPTED PENSION FUNDS (Permanent agent:  Hong Kong and Shanghai Banking Corporation, Tokyo Branch)")

0.8951517114782421